In [2]:
import string
import random
import numpy as np
import scipy as sp
import urllib.request
import nltk
import re
import heapq
from bs4 import BeautifulSoup
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
def get_article_text(url):
    # Scrape article using bs4 to extract all paragraphs from the online article.
    raw_html = urllib.request.urlopen(url)
    raw_html = raw_html.read()

    article_html = BeautifulSoup(raw_html, 'lxml')
    article_paragraphs = article_html.find_all('p')

    # Creating a document 'article_text' containing all the sentences in the article.
    article_text = ''
    for para in article_paragraphs:
        article_text += para.text
    return article_text

In [4]:
article_text = get_article_text('https://en.wikipedia.org/wiki/India')

In [5]:
def clean_sentence(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'\W', ' ', sentence)
    sentence = re.sub(r'\s+', ' ', sentence)
    return sentence

In [6]:
def clean_article_text(article_text):
    # Creating a corpus containing all the sentence tokens in the document.
    corpus = nltk.sent_tokenize(article_text)
    # Convert to lowercase, remove non-word characters (punctuations, etc.) and strip whitespaces
    for i in range(len(corpus)):
        corpus[i] = clean_sentence(corpus[i])
    return corpus

In [7]:
# Tokenize article text into sentences.
article_sentences = nltk.sent_tokenize(article_text)

# Clean the article sentence to remove extra whitespaces and reference numbers (such as "[23]")
for i in range(len(article_sentences)):
    article_sentences[i] = re.sub(r'\[\d+\]', '', article_sentences[i])
    article_sentences[i] = re.sub(r'\[\d+,\s\d+]', '', article_sentences[i])
    article_sentences[i] = re.sub(r'\[\w\]', '', article_sentences[i])
    article_sentences[i] = re.sub(r'\s+', ' ', article_sentences[i]).strip()

In [8]:
val_article_sentences = article_sentences[::4][:-11]
len(val_article_sentences)

100

In [43]:
val_answers = dict()
#val_questions = dict()

In [10]:
'''
for i, sent in enumerate(val_article_sentences):
    val_answers[i] = sent
    print('\nSentence {}: {}'.format(i, sent))
    ques = input('Enter question: ')
    val_questions[i] = ques
'''

"\nfor i, sent in enumerate(val_article_sentences):\n    val_answers[i] = sent\n    print('\nSentence {}: {}'.format(i, sent))\n    ques = input('Enter question: ')\n    val_questions[i] = ques\n"

In [12]:
#len(val_questions)

In [21]:
val_answers = dict()
for i, sent in enumerate(val_article_sentences):
    val_answers[i] = sent

In [22]:
import json
with open('val_answers.json', 'w') as fp:
    json.dump(val_answers, fp)

In [49]:
import json

with open('val_questions.json', 'w') as fp:
    json.dump(val_questions, fp)
    


Example

In [37]:
# these functions are heavily influenced by the HF squad_metrics.py script
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    print(pred_tokens)
    print(truth_tokens)
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    print(common_tokens)
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [38]:
pred = 'The ancient Greeks referred to the Indians as Indoi (Ἰνδοί), which translates as "The people of the Indus".'

In [39]:
truth = 'The ancient Greeks referred to the Indians as Indoi (Ἰνδοί), which translates as "The people of the Indus".'

In [40]:
compute_f1(pred, truth)

['ancient', 'greeks', 'referred', 'to', 'indians', 'as', 'indoi', 'ἰνδοί', 'which', 'translates', 'as', 'people', 'of', 'indus']
['ancient', 'greeks', 'referred', 'to', 'indians', 'as', 'indoi', 'ἰνδοί', 'which', 'translates', 'as', 'people', 'of', 'indus']
{'ancient', 'translates', 'which', 'to', 'people', 'ἰνδοί', 'referred', 'as', 'indus', 'indoi', 'of', 'greeks', 'indians'}


0.9285714285714286

In [41]:
compute_exact_match(pred, truth)

1